---
## Extracción de datos: API de Spotify
---

Se usa la API de Spotify para obtener información sobre canciones, géneros y artistas, en un periodo de tiempo definido.

LA API de Spotify requieren autenticación mediante claves API, tokens de acceso que puedes conseguir en la documentación de la API disponible aquí: [Spotify Web API](https://developer.spotify.com/documentation/web-api)

In [4]:
# se debe instalar la libreria 'spotipy' que sirve de puente entre la API de Spotify y el notebook

!pip install spotipy

In [1]:
# importamos las librerias necesarias para la extracción de data

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

# importamos las librerias necesarias para la conexión con SQL para la creación de la BBDD y realizar consultas en MySQL

import pandas as pd
import mysql.connector
from mysql.connector import errorcode

In [2]:
# despues de revisar la documentacion de la API de Spotify se usa el Client_ID y el Client_Secret

CLIENT_ID = 'e72fce80a01a461cbc184eae381f0700' # reemplazar por 'inserta tu Client_ID aquí'
CLIENT_SECRET = 'cc179290604c436c8b35a324eb91f00b' # reemplazar por 'inserta tu Client_Secret aquí'

# iniciamos la autenticación de la API de Spotify

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
# para la muestra de extracción seleccionamos 4 géneros y el rango de años

genres = ['pop', 'rock', 'jazz', 'r&b']
start_year = 2020
finish_year = 2022

In [ ]:
# se define una función para la extracción de las canciones con los parámetros: géneros, año_inicio, año_final, distanciamiento)

artists = [] # lista vacia definida como variable externa que se usara en la extracción usando la API de Last.fm

def extraction_songs(genres, start_year, finish_year, offsets):
    results = [] # listas vacias como variables locales
    offsets = 0
    
    for genre in genres:
        for i in range(start_year, finish_year):
            for offsets in range(0,300, 50): # se selecciona una muestra de 300 canciones por año
                data = sp.search(q=f"genre:{genre}, year:{i}", type='track', limit=50, offset=offsets)
                print(f'Retriving year {i} and offset {offsets}') # print de control para saber que información se extrayendo
                len(data)
                for item in data["tracks"]["items"]:
                    results.append({
                    'artist':item['artists'][0]['name'],
                    'genre':genre,
                    'song_name':item['name'],
                    'release_year':item['album']['release_date'][:4],
                    'id_song':item['id']})
                    artists.append(item['artists'][0]['name'])
    
    df = pd.DataFrame(results)
    print(f'format data frame songs: {df.shape}')
    df.to_csv(f'../data/songs.csv') # se crea un archivo CSV con la información de las canciones

    set_artists = set(artists)
    dfa = pd.DataFrame(set_artists)
    print(f'format data frame artists: {dfa.shape}')
    dfa.to_csv(f'../data/artists.csv') # se crea un archivo CSV con la información de los artistas que usamos en la extraccióm con la API de Last.fm

In [5]:
# se llama a la función alterando los parámetros deseados

extraction_songs(genres,2020,2022,0)

Retriving year 2020 and offset 0
Retriving year 2020 and offset 50
Retriving year 2020 and offset 100
Retriving year 2020 and offset 150
Retriving year 2020 and offset 200
Retriving year 2020 and offset 250
Retriving year 2021 and offset 0
Retriving year 2021 and offset 50
Retriving year 2021 and offset 100
Retriving year 2021 and offset 150
Retriving year 2021 and offset 200
Retriving year 2021 and offset 250
Retriving year 2020 and offset 0
Retriving year 2020 and offset 50
Retriving year 2020 and offset 100
Retriving year 2020 and offset 150
Retriving year 2020 and offset 200
Retriving year 2020 and offset 250
Retriving year 2021 and offset 0
Retriving year 2021 and offset 50
Retriving year 2021 and offset 100
Retriving year 2021 and offset 150
Retriving year 2021 and offset 200
Retriving year 2021 and offset 250
Retriving year 2020 and offset 0
Retriving year 2020 and offset 50
Retriving year 2020 and offset 100
Retriving year 2020 and offset 150
Retriving year 2020 and offset 200


---
## Extracción de datos: API de Last.fm
---

Se usa la API de Last.fm para completar la información sobre la biografía de los artistas, popularidad y artistas similares, usando la extracción de Spotify.

LA API de Last.fm requieren autenticación mediante clave API que puedes conseguir en la documentación de la API disponible aquí: [Last.fm Web API](https://www.last.fm/api)

In [49]:
# despues de revisar la documentación de la API de Last.fm se usa la API_Key y el URL

API_KEY = 'daf9398fbb92abf81e906df604450de3' # reemplazar por 'inserta tu Client_ID aquí'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

In [50]:
# Comentario Mar: crear una función? 
# usaremos la lista de artistas que extraimos de Spotify para complementar la información

set_artists_global = set(artists) # convertimos la lista en set para que elimine los artistas repetidos
list_artists = list(set_artists_global) # convertimos en lista de nuevo

---
### Extracción de información complementaria de los artistas
---

In [51]:
# función que extrae el texto limpio de la biografía

def clean_bio(bio):
    soup = BeautifulSoup(bio, "html.parser")
    return soup.get_text().strip()

# se define una función para la extracción de la información complementaria con los parámetros: lista de artistas y la autenticación de la API

def extraction_artist(list_artists, BASE_URL, API_KEY):
    artist_data = []

    for i in list_artists:
        params_info = {
            'method': 'artist.getinfo',
            'artist': i,
            'api_key': API_KEY,
            'format': 'json'
            }
        response_info = requests.get(BASE_URL, params=params_info)
        data_info = response_info.json()

        try:
            bio_summary = data_info['artist'].get('bio', {}).get('summary', 'No data')
            bio_summary = clean_bio(bio_summary)
            if not bio_summary or bio_summary == '' or bio_summary == 'No data':
                bio_summary = 'No data'
        except KeyError:
            bio_summary = 'No data'

        try:
            listeners = data_info['artist'].get('stats', {}).get('listeners', 'No data')
            if 'stats' == None:
                listeners = 'No data'
        except KeyError:
            listeners = 'No data'

        try:
            play = data_info['artist'].get('stats', {}).get('playcount', 'No data')
            if 'stats' == None:
                play = 'No data'
        except KeyError:
            play = 'No data'

        artist_data.append({
            'artist': i,
            'biography': bio_summary,
            'listeners': listeners,
            'playcount': play
        })

    df_last = pd.DataFrame(artist_data)
    print(f'format data frame artists: {df_last.shape}')
    print(df_last.head()) # print de control para saber que data estamos extrayendo

    df_last.to_csv(f'../data/artists_data.csv') # crea un archivo CSV con la información de los artistas

In [9]:
# se llama a la función con los parámetros necesarios

extraction_artist(list_artists, BASE_URL, API_KEY)

format data frame artists: (1555, 4)
       artist                                          biography listeners  \
0         BBT  BBT (Boom Boom Taker) is a four-member Chinese...      2368   
1  SaraoMusic                               Read more on Last.fm      5908   
2        Yoro  YORO born as Mitsui Ryo (三井瞭) on July 15th, 19...       312   
3      d4crvz  Gabriel da Cruz is a rapper who work in the un...       467   
4     Duocane                               Read more on Last.fm       125   

  playcount  
0     17708  
1     47822  
2      2673  
3      3732  
4       711  


---
### Extración de los artistas similares
---

In [52]:
# función que extrae artistas similares, que usa como parámetros la lista de artistas previamente guardada en la lista

def extraction_similar_artist(list_artists, BASE_URL, API_KEY):
    similar_artists = []
    
    for i in list_artists:
        params_similar = {
            'method': 'artist.getsimilar',
            'artist': i,
            'api_key': API_KEY,
            'format': 'json'
            }
        response_similar = requests.get(BASE_URL, params=params_similar)
        data_similar = response_similar.json()

        try:
            for artist in data_similar['similarartists']['artist'][0:5]:
                similar_artists.append((i, artist['name']))
        except KeyError:
            similar_artists.append((i, 'No data'))

        time.sleep(0.2)

    df_similar = pd.DataFrame(similar_artists)
    print(f'format data frame similar artists: {df_similar.shape}')
    print(df_similar.head()) # print que controla la información que estamos extrayendo

    df_similar.to_csv(f'../data/similar_artist_data.csv') # creamos un archivo CSV con la información extraída

In [53]:
# se llama a la función con los parámetros necesarios

extraction_similar_artist(list_artists, BASE_URL, API_KEY)

format data frame similar artists: (7365, 2)
     0           1
0  BBT    Awaken-F
1  BBT  TEAM SPARK
2  BBT       C.T.O
3  BBT        oner
4  BBT    Mr.Tyger


---
## Conexión con MySQL: Creación de Base de Datos (BBDD)
---

Creación de los scripts de Python/SQL para crear la base de datos y las tablas e insertar datos en la BBDD.

In [ ]:
# creación de la BBDD

def create_database():
    #MySQL connection
    cnx = mysql.connector.connect(user='root', 
                                  password='AlumnaAdalab',
                                  host='127.0.0.1')
    
    mycursor = cnx.cursor()
    try:
        mycursor.execute("CREATE DATABASE IF NOT EXISTS db_cozy_music")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    finally:
        # cierra cursor y conexión
        if 'mycursor' in locals() and mycursor is not None:
            mycursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            print("Conexión a la base de datos cerrada.")

In [ ]:
create_database()

CMySQLCursor: CREATE DATABASE IF NOT EXISTS db_cozy_mu..
Conexión a la base de datos cerrada.


---
### Creación de la tablas 'Artistas' , 'Canciones' y 'Artistas Similares'
---


In [38]:
def mysql_conn():
    return mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='db_cozy_music')

In [ ]:
# Tabla artist: incluye información del artista: ID del artista, nombre del artista, biografia, conteo de oyentes, conteo de reproducciones
# Tabla songs: incluye información del artista: ID de la canción, nombre de la canción, ID del artista, género, año de lanzamiento
# Tabla similar_artists: incluye información del artista: ID del artista, artistas similares

def create_tables():
    cnx = mysql_conn()
    mycursor = cnx.cursor()
    try:
        mycursor.execute("""
                        CREATE TABLE artist (
                        id_artist INT PRIMARY KEY AUTO_INCREMENT, 
                        artist_name VARCHAR(255), 
                        biography TEXT, 
                        listeners INT, 
                        playcount INT)""")
                        
        mycursor.execute("""
                        CREATE TABLE songs (
                        id_song VARCHAR(100) PRIMARY KEY,
                        song_name VARCHAR(255), 
                        id_artist INT, 
                        genre VARCHAR(50),
                        release_year INT,
                        FOREIGN KEY (id_artist) REFERENCES artist(id_artist) 
                        )""")
        
        mycursor.execute("""
                        CREATE TABLE similar_artists (
                        id_primary_artist INT, 
                        similar_artist VARCHAR(255),
                        FOREIGN KEY (id_primary_artist) REFERENCES artist(id_artist)
                        )""")
        
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    finally:
        # cierra cursor y conexión
        if 'mycursor' in locals() and mycursor is not None:
            mycursor.close()
        if 'cnx' in locals() and cnx.is_connected():
            cnx.close()
            print("Conexión a la base de datos cerrada.") 

In [41]:
create_tables()

CMySQLCursor: 
                        CREATE TABLE si..
Conexión a la base de datos cerrada.


---
### Inserción de data en la tabla 'Artistas'
---

In [42]:
def insert_artist_data(df):
    conn = mysql_conn()
    cursor = conn.cursor()

    # 3. insertar los datos fila por fila
    for _, row in df.iterrows():
        # limpieza: convierte 'No data' a None para listeners y playcount
        listeners = None if row['listeners'] == 'No data' else row['listeners']
        playcount = None if row['playcount'] == 'No data' else row['playcount']
        sql = """
        INSERT INTO artist (artist_name, biography, listeners, playcount)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(sql, (row['artist'], row['biography'], listeners, playcount))

    conn.commit()
    cursor.close()
    conn.close()

In [44]:
# 1. leer el CSV
df = pd.read_csv('../data/artists_data.csv')

In [45]:
insert_artist_data(df)

---
### Inserción de data en la tabla 'Artistas similares'
---

In [46]:
def insert_similar_artists(df):
    conn = mysql_conn()
    cursor = conn.cursor()

    # 3. mapeo de nombre de artista a id_artist
    cursor.execute("SELECT id_artist, artist_name FROM artist")
    artist_map = {name.lower(): id_ for id_, name in cursor.fetchall()}

    # 4. prepara los datos válidos para insertar
    records_to_insert = []
    for _, row in df.iterrows():
        id_primary_artist = artist_map.get(str(row['primary_artist']).lower())
        similar_artist = row['similar_artist']
        if id_primary_artist is not None and pd.notnull(similar_artist):
            records_to_insert.append((id_primary_artist, similar_artist))
        else:
            print(f"Advertencia: No se encontró el artista '{row['primary_artist']}' en la base de datos. Fila omitida.")

    # 5. inserta todos los registros válidos de una vez
    if records_to_insert:
        sql = """
        INSERT INTO similar_artists (id_primary_artist, similar_artist)
        VALUES (%s, %s)
        """
        cursor.executemany(sql, records_to_insert)
        conn.commit()
        print(f"{cursor.rowcount} filas insertadas en similar_artists.")
    else:
        print("No hay registros válidos para insertar.")

    cursor.close()
    conn.close()

In [55]:
# 1. leer el CSV (sin encabezados, asignando nombres)
dfs = pd.read_csv('../data/similar_artist_data.csv', header=None, names=['primary_artist', 'similar_artist'])

In [56]:
insert_similar_artists(dfs)

Advertencia: No se encontró el artista '0' en la base de datos. Fila omitida.
7365 filas insertadas en similar_artists.


---
### Inserción de data en la tabla 'Canciones'
---

In [57]:
def insert_songs(df):
    conn = mysql_conn()
    cursor = conn.cursor()

    # 3. mapeo de nombre de artista a id_artist
    cursor.execute("SELECT id_artist, artist_name FROM artist")
    artist_map = {name.lower(): id_ for id_, name in cursor.fetchall()}

    # 4. prepara los datos válidos para insertar
    records_to_insert = []
    for _, row in df.iterrows():
        id_song = row['id_song']
        song_name = row['song_name']
        artist_name = str(row['artist']).lower()
        id_artist = artist_map.get(artist_name)
        genre = row['genre']
        release_year = row['release_year']

        # validaciones básicas
        if id_artist is not None and pd.notnull(id_song) and pd.notnull(song_name):
            records_to_insert.append((id_song, song_name, id_artist, genre, release_year))
        else:
            print(f"Advertencia: No se encontró el artista '{row['artist']}' o datos faltantes en la fila. Fila omitida.")

    # 5. inserta todos los registros válidos por lotes

    if records_to_insert:
        sql = """
        INSERT IGNORE INTO songs (id_song, song_name, id_artist, genre, release_year)
        VALUES (%s, %s, %s, %s, %s)
        """
        batch_size = 500  # se puede ajustar el tamaño según la RAM
        for i in range(0, len(records_to_insert), batch_size):
            batch = records_to_insert[i:i+batch_size]
            cursor.executemany(sql, batch)
            conn.commit()
        print(f"{len(records_to_insert)} filas insertadas en songs.")
    else:
        print("No hay registros válidos para insertar.")

    cursor.close()
    conn.close()

In [58]:
# 1. leer el CSV
dfso = pd.read_csv('../data/songs.csv')

In [59]:
insert_songs(dfso)

2400 filas insertadas en songs.


---
## Conexión con MySQL: Consultas en la Base de Datos (BBDD)
---

Ejecutar consultas SQL para responder a preguntas específicas que luego analizaremos.

In [60]:
#-- I. ¿Cuál es el artista con más popularidad y a qué género pertenece? 
#-- Tomando listeners (seguidores) como medida de popularidad (listeners y playcount colocan a Coldplay de primero)
def most_popular_artist():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query de ejemplo:
    query = ("""SELECT a.artist_name, s.genre
                FROM songs s
                INNER JOIN artist a USING(id_artist)
                WHERE id_artist = (
                    SELECT id_artist
                    FROM artist
                    ORDER BY listeners DESC
                    LIMIT 1
                );
                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    print(f'The most popular artist is: {df_results.iloc[0, 0]}, and the genre is: {df_results.iloc[0, 1]}.')

In [61]:
most_popular_artist()

The most popular artist is: The Doors, and the genre is: rock.


In [68]:
#-- II. ¿En qué año se lanzaron más canciones, 2020 o 2021?

def most_songs_year():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query de ejemplo:
    query = ("""SELECT release_year, COUNT(id_song) AS total_songs
                FROM songs
                GROUP BY release_year
                ORDER BY total_songs DESC
                LIMIT 1;
                            );
                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)

    print(f'Year with most songs released: {df_results.iloc[0, 0]}')
    print(f'Total songs released in {df_results.iloc[0, 0]}: {df_results.iloc[0, 1]}')

In [69]:
most_songs_year()

Year with most songs released: 2020
Total songs released in 2020: 1104


In [70]:
#-- III. ¿Cuántas canciones y artistas tenemos en nuestra BBDD? 

def total_songs_artists():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query de ejemplo:
    query = ("""SELECT COUNT(DISTINCT s.id_song) AS total_songs,
                COUNT(DISTINCT a.id_artist) AS total_artists
                FROM songs s
                JOIN artist a USING(id_artist);
                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)

    print(f'The total number of songs in our database is {df_results.iloc[0, 0]}')
    print(f'And the total number of artists is {df_results.iloc[0, 1]}')


In [71]:
total_songs_artists()

The total number of songs in our database is 2191
And the total number of artists is 1555


In [74]:
#-- IV. Ranking géneros con más oyentes y reproducciones de mayor a menor.

def genre_ranking():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query de ejemplo:
    query = ("""SELECT genre, SUM(listeners) AS total_listeners, SUM(playcount) AS total_playcount
                    FROM artist
                    INNER JOIN songs USING(id_artist)
                    GROUP BY genre
                    ORDER BY total_listeners DESC;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    results
    df_results = pd.DataFrame(results)
    df_results.columns = ['Genre', 'Total Listeners', 'Total Playcount']
    return df_results

In [75]:
genre_ranking()

,Genre,Total Listeners,Total Playcount
0,rock,179173938,3783676305
1,r&b,87443481,1307078762
2,pop,72240209,1472046695
3,jazz,41773152,497608759


In [76]:
#-- V. Top 5 de artistas más escuchados de cada género y su biografía y artistas similares.

def top_5_artists_genre():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query:
    query = ("""SELECT 
                    a.artist_name,
                    s.genre,
                    CAST(COALESCE(a.listeners, 0) AS UNSIGNED) AS total_listeners,
                    a.biography,
                    GROUP_CONCAT(DISTINCT sa.similar_artist SEPARATOR ', ') AS similar_artists
                FROM artist a
                INNER JOIN songs s ON a.id_artist = s.id_artist
                LEFT JOIN similar_artists sa ON sa.id_primary_artist = a.id_artist
                WHERE
                    (
                        SELECT COUNT(DISTINCT a2.id_artist)
                        FROM artist a2
                        INNER JOIN songs s2 ON a2.id_artist = s2.id_artist
                        WHERE s2.genre = s.genre AND a2.listeners > a.listeners
                    ) < 5
                GROUP BY a.artist_name, s.genre, a.listeners, a.biography
                ORDER BY s.genre, a.listeners DESC;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    df_results.columns = ['Artist Name', 'Genre', 'Total Listeners', 'Biography', 'Similar Artists']
    return df_results

In [77]:
top_5_artists_genre()

,Artist Name,Genre,Total Listeners,Biography,Similar Artists
0,Louis Armstrong,jazz,2712606,"Louis Armstrong (August 4, 1901 - July 6, 1971...","Benny Goodman, Duke Ellington, Ella Fitzgerald..."
1,Ella Fitzgerald,jazz,2642994,"Ella Jane Fitzgerald (April 25, 1917 – June 15...","Billie Holiday, Dinah Washington, Ella Fitzger..."
2,Dean Martin,jazz,2567892,"Dean Martin (born Dino Paul Crocetti; June 7, ...","Bobby Darin, Frank Sinatra, Nat King Cole, Per..."
3,Ray Charles,jazz,2504622,"Ray Charles (Ray Charles Robinson, Albany, Ge...","Aretha Franklin, Etta James, Louis Armstrong, ..."
4,Otis Redding,jazz,2190440,"Otis Ray Redding Jr. (September 9, 1941 – Dece...","Otis Redding & Carla Thomas, Percy Sledge, Sam..."
5,Hank Harpers Trio,jazz,0,No data,No data
6,Lorenzo Luca,jazz,0,No data,No data
7,The Sleepy Bears,jazz,0,No data,No data
8,Flo Rida,pop,3750898,"Tramar Dillard (born September 17, 1979 in Mia...","B.o.B, Far East Movement, Pitbull, Sean Kingst..."
9,Kylie Minogue,pop,3152218,"Kylie Ann Minogue is an Australian singer, son...","Cher, Dannii Minogue, Girls Aloud, Madonna, So..."


In [78]:
#-- VI. ¿Qué canciones tienen distintas versiones en la BBDD y en qué géneros aparecen?

def different_versions_songs():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query:
    query = ("""SELECT
                    s.song_name,
                    GROUP_CONCAT(DISTINCT a.artist_name SEPARATOR ', ') AS artist,
                    GROUP_CONCAT(DISTINCT s.genre SEPARATOR ', ') AS genre
                FROM songs AS s
                INNER JOIN artist AS a USING(id_artist)
                GROUP BY s.song_name
                HAVING COUNT(DISTINCT s.genre) > 1;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    df_results.columns = ['Song Name', 'Artist Name', 'Genre']
    return df_results

In [79]:
different_versions_songs()

,Song Name,Artist Name,Genre
0,All My Life,"Ivory Joe Hunter, James Moody, Sweaty Palms","jazz, rock"
1,Batalha,Carranca,"r&b, rock"
2,Can't Find My Way Home,Bonnie Raitt,"r&b, rock"
3,"December, 1963 (Oh What a Night!)",Frankie Valli & The Four Seasons,"pop, r&b"
4,I Can't Tell the Bottom from the Top,The Hollies,"r&b, rock"
5,I Get a Kick Out of You,"Bobby Solo, Matt Belsante","jazz, pop"
6,Intro,"Best Intentions, Hollow Da Don, Pihlanne, Taweh G","jazz, pop, r&b"
7,It's Christmas,Pilot,"pop, rock"
8,Lady,"Eric Benét, Jay Jay","pop, r&b"
9,Little Darlin',"Jimmy Heath, Tommy Steele","jazz, rock"


In [80]:
#-- VII. ¿Qué artista tiene más canciones en la BBDD?

def most_songs_artist():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query:
    query = ("""SELECT
                    artist_name,
                    COUNT(id_song) AS total_songs
                FROM artist
                INNER JOIN songs USING(id_artist)
                GROUP BY artist_name
                HAVING total_songs >1
                ORDER BY total_songs DESC
                LIMIT 1;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)

    print(f'The artist with the most songs in our database is {df_results.iloc[0, 0]} with {df_results.iloc[0, 1]} songs ')

In [81]:
most_songs_artist()

The artist with the most songs in our database is Late Night Jazz Lounge with 18 songs 


In [82]:
#-- VIII. ¿Cuántas canciones por género tenemos en nuestra base de datos?

def songs_by_genre():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query:
    query = ("""SELECT genre, COUNT(DISTINCT id_song) AS total_songs
                FROM songs
                GROUP BY genre
                ORDER BY total_songs DESC;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    df_results.columns = ['Genre', 'Total Songs']
    return df_results

In [83]:
songs_by_genre()

,Genre,Total Songs
0,jazz,564
1,pop,559
2,rock,543
3,r&b,525


In [84]:
#-- IX. ¿Qué artistas aparecen más veces como "artista similar" de otros artistas?

def most_similar_artists():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3. query:
    query = ("""SELECT sa.similar_artist, COUNT(*) AS times_similar
                FROM similar_artists sa
                GROUP BY sa.similar_artist
                ORDER BY times_similar DESC
                LIMIT 10;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    df_results.columns = ['Similar Artist', 'Times Similar']
    return df_results

In [85]:
most_similar_artists()

,Similar Artist,Times Similar
0,Neelkamal Singh,18
1,Khesari Lal Yadav,16
2,Shilpi Raj,14
3,Samar Singh,11
4,Monica,10
5,Java Jazz Cafe,10
6,Big Joe Turner,9
7,Lloyd Price,9
8,Wynonie Harris,8
9,Cafe Jazz Duo,8


In [86]:
#-- X.  Ranking de artistas por género, clasificados en 3 grupos según cantidad de listeners

def artist_by_genre():
    conn = mysql_conn()
    # 2. iniciamos el cursor   
    cursor = conn.cursor()

    # 3.query:
    query = ("""SELECT
                artist_name,
                genre,
                listeners,
                group_popularity          
                FROM (
                SELECT
                    genre,
                    artist_name,
                    listeners,
                    group_popularity,
                    ROW_NUMBER() OVER (
                        PARTITION BY genre, group_popularity
                        ORDER BY listeners DESC
                    ) AS rn
                FROM (
                    SELECT
                        s.genre,
                        a.artist_name,
                        MAX(a.listeners) AS listeners,  -- O SUM(a.listeners) si quieres sumar
                        CASE
                            WHEN MAX(a.listeners) >= 50000 THEN 'Most listened'
                            WHEN MAX(a.listeners) >= 10000 THEN 'Average'
                            ELSE 'Least listened'
                        END AS group_popularity
                    FROM songs s
                    JOIN artist a ON s.id_artist = a.id_artist
                    GROUP BY s.genre, a.artist_name
                ) grouped
            ) ranked
            WHERE rn <= 3
            ORDER BY genre, group_popularity, listeners DESC;
                                                """)

    # 4. luego llamamos al método execute() del cursor, al que pasamos como argumento el string que contiene la consulta.
    # De esta manera se ejecuta la consulta en la base de datos y si todo va correctamente, el resultado se almacenará en el cursor.
    cursor.execute(query)

    results = cursor.fetchall()
    cursor.close()
    conn.close()

    df_results = pd.DataFrame(results)
    df_results.columns = ['Artist Name','Genre', 'Listeners' ,'Group Popularity']
    return df_results

In [87]:
artist_by_genre()

,Artist Name,Genre,Listeners,Group Popularity
0,Ellis Marsalis,jazz,49411,Average
1,Chuck Willis,jazz,47227,Average
2,Scrimshire,jazz,46339,Average
3,Jazz For Sleeping,jazz,9921,Least listened
4,Bruno Spoerri,jazz,9696,Least listened
5,Victor Feldman,jazz,9627,Least listened
6,Louis Armstrong,jazz,2712606,Most listened
7,Ella Fitzgerald,jazz,2642994,Most listened
8,Dean Martin,jazz,2567892,Most listened
9,Connie Smith,pop,48374,Average
